In [1]:
import glm
import numpy as np
import pygame
import moderngl
from math import cos, sin, sqrt
import numpy
from loadModelUsingAssimp_V3 import create3DAssimpObject
import ctypes
# ctypes.windll.user32.SetProcessDPIAware()

pygame 2.6.1 (SDL 2.28.4, Python 3.13.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


/Users/rafa/Developer/GRAPHICS/.cgenv/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


### Initialize pygame and create a window with OpenGL context.

In [2]:
width = 840
height = 480

pygame.init() # Initlizes its different modules. Display module is one of them.
pygame.display.gl_set_attribute(pygame.GL_MULTISAMPLEBUFFERS, 1)  #pygame.display.gl_set_attribute(pygame.GL_STENCIL_SIZE, 8)
pygame.display.gl_set_attribute(pygame.GL_MULTISAMPLESAMPLES, 16)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE) 
pygame.display.set_mode((width, height), flags= pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title = "Class Practice: <your name>")
gl = moderngl.get_context() # Get previously created context.
gl.info["GL_VERSION"]
FB = gl.detect_framebuffer()

### [Todo] Create Offscreen Frame Buffer 

### Read Model and create Floor

In [3]:
modelFile = "chair_table_class/scene.gltf" 
modelObj = create3DAssimpObject(modelFile, verbose=False, textureFlag = True, normalFlag = True)
bound = modelObj.bound

# Base Plane Center of the bounding box:
# The extent of base plane parallel to XY plane
_minP = bound.boundingBox[0]
_maxP = glm.vec3(bound.boundingBox[1].x, _minP.y,bound.boundingBox[1].z) 
_center = (_minP + _maxP)/2

planePoint = _center
planeNormal = glm.vec3(0,1,0)

squareQuadSide = 3*bound.radius
halfSideSize = squareQuadSide/2
baseQuadGeomBuffer = gl.buffer(numpy.array([
    _center.x - halfSideSize, _center.y, _center.z - halfSideSize, 0, 1, 0, 0, 0,
    _center.x + halfSideSize, _center.y, _center.z - halfSideSize, 0, 1, 0, 1, 0,
    _center.x + halfSideSize, _center.y, _center.z + halfSideSize, 0, 1, 0, 1, 1,
    _center.x - halfSideSize, _center.y, _center.z + halfSideSize, 0, 1, 0, 0, 1
]).astype("float32"))
_index = numpy.array([
    0, 1, 2,
    2, 3, 0
]).astype("int32")
baseQuadIndexBuffer = gl.buffer(_index)

_wood_texture_img = pygame.image.load("tile-squares-texture.jpg") 
_texture_data = pygame.image.tobytes(_wood_texture_img,"RGB", True) 
_texture = gl.texture(_wood_texture_img.get_size(), data = _texture_data, components=3)
_texture.build_mipmaps()
floor_sampler = gl.sampler(texture=_texture, filter=(gl.LINEAR_MIPMAP_LINEAR, gl.LINEAR), repeat_x = True, repeat_y = True)

Accumulates Normals in the geomDataList
Accumulates Textures in the geomDataList


### Renderer: Used for both Model object and Floor rendering. 
Modify the shader code (both vertex and fragment shader code) to carry out rendering from light to write to the Depth buffer of your own Frame buffer, and to render shadow using shadomap.

In [4]:
def queryProgram(program):
    for name in program:
        member = program[name]
        print(name, type(member), member)
        
#
# Vertex shader(s)
#
vertex_shader_code = '''
#version 410 core
layout (location=0) in vec3 position;
layout (location=1) in vec3 normal;
layout (location=2) in vec2 uv;


uniform mat4 model, view, perspective;

out vec2 f_uv;
out vec3 f_normal; // Normal vector in World Coordinates
out vec3 f_position; // postion in world coordinates

void main() {
    vec4 P = model*vec4(position, 1);
    f_position = P.xyz;

    f_uv = uv;
    mat3 normalMatrix = mat3(transpose(inverse(model)));// inverse transpose of model transformation
    f_normal = normalize(normalMatrix*normal);

    gl_Position = perspective*view*P;
 }
'''

#
# Fragment shader(s)
#
fragment_shader_code = '''
    #version 410 core
    in vec2 f_uv;
    in vec3 f_normal;
    in vec3 f_position;
    
    uniform sampler2D map;

    uniform bool biasFlag;
    uniform int pcf;
    
    uniform vec3 light;
    
    uniform float shininess;
    uniform vec3 eye_position;
    uniform vec3 k_diffuse;
    
    const vec3 up = vec3(0, 1, 0);
    const vec3 groundColor = vec3(0.3215686274509804,0.4,0.10980392156862745);
        const vec3 skyColor = vec3(0.7176470588235294, 0.7411764705882353, 0.7529411764705882);

    // Add output variable here
    out vec4 out_color;
    
    float ComputeVisibilityFactor(){ // Modify this function to compute visibility (or fractional visibility for PCF support) by accessing shadowmap
        return 1.0;
    }
    
    vec3 computeColor(){
        vec3 L = normalize(light.xyz-f_position);
        vec3 materialColor = texture(map, f_uv).rgb;
        vec3 N = normalize(f_normal);
        float NdotL = dot(N,L);
        vec3 color = vec3(0.);
        float w = dot(N,up);
        vec3 ambientColor = 0.25*(w*skyColor+(1-w)*groundColor)*materialColor;
        float fractionalLightVisibility = ComputeVisibilityFactor();
        if (NdotL>0.){
            vec3 diffuselyReflectedColor = materialColor * NdotL;
            // Compute specular color
            vec3 V = normalize(eye_position - f_position);
            vec3 H = normalize(L+V);
            vec3 specularlyReflectedColor = vec3(0);
            if (shininess > 0)
                specularlyReflectedColor = vec3(1)*pow(dot(N,H), shininess);
            color = fractionalLightVisibility*(k_diffuse * diffuselyReflectedColor + specularlyReflectedColor);
        }
        color += ambientColor; 
        return color;
    }
    void main() {
        out_color = vec4(computeColor(), 1);
    }
'''
#
# Programs
#

program = gl.program(
    vertex_shader= vertex_shader_code,
    fragment_shader= fragment_shader_code
)

queryProgram(program)

#model_renderables = modelObj.getRenderables(gl, model_program, format, variables)
modelObj.createRenderableAndSampler(program)

floorRenderer = gl.vertex_array(
    program,
    [(baseQuadGeomBuffer, "3f 3f 2f", "position", "normal", "uv")],
    baseQuadIndexBuffer,index_element_size=4
)

def render_model():
    modelObj.render()
def render_floor():
    floor_sampler.use(0)
    program["model"].write(glm.mat4(1))
    floorRenderer.render()
def renderScene(viewMatrix, perspectiveMatrix, light, eye):
    program["view"].write(viewMatrix)
    program["perspective"].write(perspectiveMatrix)   
    program["eye_position"].write(eye)
    program["light"].write(light)
    render_model()
    program["shininess"] = 0
    render_floor()

uv <class '_moderngl.Attribute'> <Attribute: 2>
normal <class '_moderngl.Attribute'> <Attribute: 1>
position <class '_moderngl.Attribute'> <Attribute: 0>
shininess <class '_moderngl.Uniform'> <Uniform: 0>
perspective <class '_moderngl.Uniform'> <Uniform: 1>
model <class '_moderngl.Uniform'> <Uniform: 5>
map <class '_moderngl.Uniform'> <Uniform: 9>
light <class '_moderngl.Uniform'> <Uniform: 10>
view <class '_moderngl.Uniform'> <Uniform: 11>
eye_position <class '_moderngl.Uniform'> <Uniform: 15>
k_diffuse <class '_moderngl.Uniform'> <Uniform: 16>


### [TODO] Write shader proram to show shadow map in a small viewport overlapping the main window.

In [5]:
#[Todo] (i) Write a shader program to show shadowmap 
#[Todo] (ii) Create a quad geometry with texture coordinates to show the shadowmp
#[Todo] (iii) Create buffer and renderable

def showShadowMap():
    size = width/4
    gl.viewport= width-size, height-size, size, size
    gl.clear(0.5,0,0.5, viewport=gl.viewport)
    # [TODO] make render call to show the shadowmap. on this viewport
    gl.viewport=0,0,width, height

### Define Camera Parameters

In [6]:
displacement_vector = 4*bound.radius*glm.rotateX(glm.vec3(0,1,0), glm.radians(85)) #glm.vec3(0,0,1) 

light_displacement_vector = 4*bound.radius*glm.rotateZ(glm.vec3(0,1,0), glm.radians(45)) 
    
target_point = glm.vec3(bound.center)
up_vector = glm.vec3(0,1,0)

### View volume parameters
fov_radian = glm.radians(30) # In radian
aspect = width/height
near = bound.radius
far = 20*bound.radius
perspectiveMatrix = glm.perspective(fov_radian, aspect, near, far)

In [7]:
running = True
clock = pygame.time.Clock()
alpha = 0
lightAngle = 0
pcf = 0


pause = True   # Keyboard key "p" toggles pause/orbit
debug = False # Keyboard key "d" toggles display of shadowmap in a quarter-size viewport place at the right.
bias = True  # Keyboard key "b" toggles bias/no bias
print(" Camera Orbiting Paused. No Shadow. Point Light. Bias. PCF : ", pcf)
#gl.depth_func = '<=' 
gl.enable(gl.DEPTH_TEST)
while running:   
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif (event.type ==  pygame.KEYDOWN):
            if  event.key == 27:
                running = False
            elif event.key == pygame.K_p:
                pause = not pause
                print("Camera Orbiting" + " paused" if pause else "")
            elif event.key == pygame.K_d:
                debug = not debug
                print("Shadowmap" + " visible" if debug else "hidden.")
            elif event.key == pygame.K_b:
                bias = not bias
                print("bias" if bias else "No bias")
            elif event.key == pygame.K_LEFT:
                lightAngle -= 5
            elif event.key == pygame.K_RIGHT:
                lightAngle += 5
        elif (event.type == pygame.WINDOWRESIZED):
            width = event.x
            height = event.y
            perspectiveMatrix = glm.perspective(fov_radian, width/height, near, far)

    
    # create the aspect ratio correction matrix
    new_displacement_vector = glm.rotateY(displacement_vector, glm.radians(alpha))

    new_light_displacement_vector = glm.rotateY(light_displacement_vector, glm.radians(lightAngle))
    
    light_point =target_point+new_light_displacement_vector
    
    
    eye_point = target_point + new_displacement_vector

    viewMatrix = glm.lookAt(eye_point, target_point, up_vector)

    gl.clear(0.2,0.2,0)
    renderScene(viewMatrix, perspectiveMatrix, light_point, eye_point)
    ##[Todo] Replace renderScene call with two passes. One to create shadomap. the other to use shadowmap.
    ##       Each pass will call the renderScene with the appropriate uniform value.

    if debug:
        showShadowMap()

    pygame.display.flip()
    clock.tick(60)  # limits FPS to 10
    if not pause:
        alpha +=  1
        if alpha > 360:
            alpha = 0
        
    #running = False
pygame.display.quit()

 Camera Orbiting Paused. No Shadow. Point Light. Bias. PCF :  0
